In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1168453109350621018
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 16152002560
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15519934931380951503
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


2021-12-14 08:26:49.201043: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-14 08:26:49.264154: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-14 08:26:49.367800: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-14 08:26:49.369008: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [2]:
import tensorflow as tf
tf.test.is_gpu_available()

2021-12-14 08:26:54.730511: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-14 08:26:54.731624: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-14 08:26:54.732629: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-14 08:26:54.733786: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-14 08:26:54.734917: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

True

In [3]:
from numpy import array
from numpy import argmax
from pandas import DataFrame
from nltk.translate.bleu_score import corpus_bleu
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Embedding
from keras.layers.merge import add
from keras.layers import Dropout
from numpy import argmax
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

In [4]:
def LoadDoc(file_name):
    file = open(file_name, 'r', encoding = "utf-8")
    text_data = file.read()
    file.close()
    return text_data


# In[14]:


def LoadPhotoFeatures(file_name, dataset):
    AllFeatures = load(open(file_name, 'rb'))
    features = {k[:-4]: AllFeatures[k[:-4]] for k in dataset}
    return features

In [5]:
def LoadPhotoFeatures_2(file_name, dataset):
    AllFeatures = load(open(file_name, 'rb'))
    #print(AllFeatures.keys())
    features = {}
    for k in dataset:
        if k.split(".")[0] in AllFeatures.keys():
            features[k.split(".")[0]] = AllFeatures[k.split(".")[0]]
    #features = { k[:-4]: AllFeatures[k[:-4]] for k in dataset}
    return features

In [6]:
def LoadCleanDescription(file_name):
    document = LoadDoc(file_name)
    descriptions = dict()
    i = 0
    for l in document.split('\n'):
        if len(l)<1:
            print(l,":",i)
            break
        i +=1
        if i == 1:
            continue
        tokens = l.split(',')
        imageId, imageDesc = tokens[0], tokens[1].split()
        if imageId not in descriptions:
            descriptions[imageId] = list()
        desc = 'startseq ' + ' '.join(imageDesc) + ' endseq'
        descriptions[imageId].append(desc)
    return descriptions

In [7]:
def ToLines(desc):
    all_desc = list()
    for key in desc.keys():
        [all_desc.append(d) for d in desc[key]]
    return all_desc


# In[17]:


def CreateTokenizer(desc):
    l = ToLines(desc)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(l)
    return tokenizer


# In[18]:


def maxLength(descs):
    lines = ToLines(descs)
    return max(len(d.split()) for d in lines)

In [8]:
def DefineModel(vocabSize, maxLength):
    inputs1 = Input(shape=(4096,))
    fe1 = Dropout(0.5, seed = 121)(inputs1)
    fe2 = Dense(128, activation='relu')(fe1)
    fe3 = RepeatVector(max_length)(fe2)
    
    # embedding
    inputs2 = Input(shape=(max_length,))
    emb2 = Embedding(vocab_size, 50, mask_zero=True)(inputs2)
    emb2a = Dropout(0.5, seed = 121)(emb2)
    emb3 = LSTM(128, return_sequences=True)(emb2a)
    emb4 = LSTM(128, return_sequences=True)(emb3)
    emb5 = TimeDistributed(Dense(128, activation='relu'))(emb4)
    # merge inputs
    merged = add([fe3, emb5])
    # language model (decoder)
    lm1 = Dropout(0.5, seed = 121)(merged)
    lm2 = LSTM(500)(lm1)
    lm3 = Dense(500, activation='relu')(lm2)
    outputs = Dense(vocab_size, activation='softmax')(lm3)
    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer = 'adam' , metrics=['accuracy'])
    print(model.summary())
    plot_model(model, show_shapes=True, to_file='plot.png')
    return model

In [9]:
def CreateSequence(tokenizer, maxLength, desc, images, vocabSize):
    X1, X2, y = list(), list(), list()
    for key, desc_list in desc.items():
        #print(key)
        for desc_sent in desc_list:
            #print(desc_sent)
            sequence = tokenizer.texts_to_sequences([desc_sent])[0]
            for i in range(1, len(sequence)):
                in_sequence, out_sequence = sequence[:i], sequence[i]
                in_sequence = pad_sequences([in_sequence], maxlen=maxLength)[0]
                out_sequence = to_categorical([out_sequence], num_classes=vocabSize)[0]
                X1.append(images[key.split('.')[0]][0])
                X2.append(in_sequence)
                y.append(out_sequence)
                # break
            #break
        #break
    return array(X1), array(X2), array(y)

In [10]:
#! cat ../input/setfiles/valid_file.tsv ../input/setfiles/train_file.tsv  > combined_train_val.csv
! cat ../input/ic-train/ic_train.csv ../input/indiancaptionspart2/ic_captions_processed_part_2.csv > combined_indian.csv

In [11]:
train_descriptions_set_2 = LoadCleanDescription('./combined_indian.csv')
train_descriptions_set_1 = LoadCleanDescription('../input/combined-train-val/combined_train_val.csv')

train_descriptions = {**train_descriptions_set_1 ,** train_descriptions_set_2}
# test_descriptions = LoadCleanDescription('/kaggle/input/setfiles/test_file.tsv')
#valid_descriptions = LoadCleanDescription('../valid_file.tsv')
train_descriptions.pop('image_file')
print('Descriptions:',len(train_descriptions))

 : 177
 : 32365
Descriptions: 6649


In [ ]:
# k1 = list(train_descriptions.keys())
# k2 = list(train_features.keys())
# for k in k1:
#     if k.split('.')[0] not in k2:
#         print(k)
        

In [12]:
train_features_set_1 = LoadPhotoFeatures_2('../input/d/sayalighodekar/vgg16features/VGG16features.pkl', train_descriptions_set_1)
train_features_set_2 = LoadPhotoFeatures_2('../input/india-vgg16/india_VGG16features.pkl', train_descriptions_set_2)
train_features = {**train_features_set_1 ,** train_features_set_2}
print("len of train feats", len(train_features))

len of train feats 6562


In [ ]:
#print('Photos: train=%d, valid=%d' % (len(train_features), len(valid_features)))

tokenizer = CreateTokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = maxLength(train_descriptions)
print('Description Length: %d' % max_length)

In [ ]:
import numpy as np
import keras

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self,ImageFeatures,tokenizer,maxLength,descriptions,vocabSize,batch_size=32):
        'Initialization'
        self.batch_size = batch_size
        self.ImageFeatures = ImageFeatures
        self.tokenizer = tokenizer
        self.maxLength = maxLength
        self.descriptions = descriptions
        self.vocabSize = vocabSize
        self.keys = list(descriptions.keys())
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.descriptions) / self.batch_size))
    
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        batch_keys = self.keys[index*self.batch_size:(index+1)*self.batch_size]
        batch_descriptions = { your_key: self.descriptions[your_key] for your_key in batch_keys }
        batch_features = { your_key.split(".")[0]: self.ImageFeatures[your_key.split(".")[0]] for your_key in batch_keys}
        X1,X2,Y = CreateSequence(self.tokenizer, self.maxLength, batch_descriptions,batch_features,self.vocabSize)
        X_final = [X1, X2]
        Y_final = Y
        return X_final, Y_final
                                 
#     def CreateSequence(tokenizer, maxLength, desc, images, vocabSize):
#         X1, X2, y = list(), list(), list()
#         for key, desc_list in desc.items():
#             #print(key)
#             for desc_sent in desc_list:
#                 #print(desc_sent)
#                 sequence = tokenizer.texts_to_sequences([desc_sent])[0]
#                 for i in range(1, len(sequence)):
#                     in_sequence, out_sequence = sequence[:i], sequence[i]
#                     in_sequence = pad_sequences([in_sequence], maxlen=maxLength)[0]
#                     out_sequence = to_categorical([out_sequence], num_classes=vocabSize)[0]
#                     X1.append(images[key[:-4]][0])
#                     X2.append(in_sequence)
#                     y.append(out_sequence)
#                     # break
#                 #break
#             #break
#         return array(X1), array(X2), array(y)

In [ ]:
training_generator = DataGenerator(train_features,tokenizer,max_length,train_descriptions,vocab_size)

In [ ]:
model = DefineModel(vocab_size, max_length)
model.fit_generator(generator=training_generator,
                    use_multiprocessing=True,
                    epochs=40, verbose=2,
                    workers=8)

In [ ]:
model.save("indian_images_vgg_captions")

In [ ]:
!zip -r ./final_image_vgg_captions.zip ./indian_images_vgg_captions